https://jamm-notnull.tistory.com/12 
- 생략한 것들은 HousePrice1에서 한번씩 다 해본 것이므로 생략했다
- 추가로 적을게 있었다면 작성함!

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings;
warnings.simplefilter('ignore')

In [2]:
# 자동완성 기능
%config Completer.use_jedi = False

In [3]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv',index_col='Id')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv',index_col='Id')
submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv',
                        index_col='Id')
print(train.shape,test.shape,submission.shape)

In [4]:
data = train

# Regression 문제인 만큼 더더욱 타켓 변수의 분포를 확인할 것
figure, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
figure.set_size_inches(14,6)
sns.distplot(data['SalePrice'], fit=norm, ax=ax1)
sns.distplot(np.log(data['SalePrice']+1), fit=norm, ax=ax2)

# Skewness : 데이터 분포의 비대칭 정도

In [5]:
# 상관관계 확인하기 0.3 이상만!
corrmat = train.corr()
top_corr_features = corrmat.index[abs(corrmat["SalePrice"])>=0.3]
top_corr_features

In [6]:
# heatmap
plt.figure(figsize=(13,10))
g = sns.heatmap(train[top_corr_features].corr(),annot=True, cmap='RdYlGn')

In [7]:
sns.regplot(data['GrLivArea'], data['SalePrice'])

# GrLivArea가 증가하면 집 값도 증가
# 오른쪽 아래 두점 : 전혀 다른 결과를 나타냄으로 이상치(Outier)로 간주하고 삭제

In [8]:
# 번외
Ytrain = train['SalePrice']
train = train[list(test)]
all_data = pd.concat((train,test), axis=0)
print(all_data.shape)
Ytrain=np.log(Ytrain+1)
# 로그를 씌우는데 +1을 해주는 이유는 log 0 라는 값이 존재하지 않기 때문에
# Ytain의 값중에 0이 존재한다면 log 0 는 마이너스 무한대로 가기 때문에 +1를 해줘서 방지해준다.

In [9]:
# 결측치 확인
cols = list(all_data)
for col in list(all_data):
    if(all_data[col].isnull().sum())==0:
        cols.remove(col)
    else:
        pass
print(len(cols))

# all_data에서 해당 열에 결측치가 없으면 리스트에서 그 열의 이름을 지운다
# 즉, 남은 리스트에는 결측치가 있는 변수만 남아 있을 것이다

In [10]:
# 결측치 채우기
# 집에 해당 시설물이 없는 경우( 범주형 변수 ), 이때 결측치를 None 이라는 문자열로 채운다
for col in ('PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageType','GarageFinish','GarageQual',
           'GarageCond','BsmtQual','BsmtExposure','BsmtFinType1','BsmtFinType2','MasVnrType','MSSubClass'):
    all_data[col] = all_data[col].fillna('None')

# 집에 해당 시설물이 없는 경우( 수치형 변수 ), 이때는 결측치를 0 으로 채운다
# 차고 면적 = 0 이면 차고 없음
for col in ('GarageYrBlt','GarageArea','GarageCars','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
           'BsmtFullBath','BsmtHalfBath','MasVnrArea','LotFrontage'):
    all_data[col] = all_data[col].fillna(0)


# 해당 시설물이 없다고 보기 힘든 경우에 있는 결측치, 이때는 결측치를 해당 열의 최빈값으로 채운다.
for col in ('MSZoning','Electrical','KitchenQual','Exterior1st','Exterior2nd','SaleType','Functional','Utilities'):
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [11]:
# 총 욕실 수 
figure, (ax5) = plt.subplots(nrows=1, ncols=1)
figure.figure.set_size_inches(14,6)
sns.barplot(data['BsmtFullBath'] + data['FullBath'] + (data['BsmtHalfBath']/2) + (data['HalfBath']/2),
           data['SalePrice'], ax=ax5)

# 검정 선은 편차를 의미한다.
# 그렇다면 편차가 없다는 것이 무슨 뜻이지?
# 값이 다른 데이터가 두개만 있어도 편차가 발생할텐데?
# 화장실 갯수가 5개, 6개인 집은 각각 하나씩 밖에 없다는 뜻으로 볼 수 있다.
# 이들은 역시 outier로 판단하고 지워도 상관없다.

In [12]:
# 기타 변수

# 1. 수영장
figure, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
figure.figure.set_size_inches(14,6)
sns.regplot(data=data, x='PoolArea', y='SalePrice', ax=ax1)
sns.barplot(data=data, x='PoolQC', y='SalePrice', ax=ax2)

# 0에 몰려있는 모습을 확인할 수 있다.

In [13]:
# 2. 테니스 코트 
figure, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
figure.figure.set_size_inches(14,6)
sns.regplot(data=data, x='MiscVal', y='SalePrice', ax=ax1)
sns.barplot(data=data, x='MiscFeature',y='SalePrice', ax=ax2)

In [14]:
# 3. 위 둘보다는 괜찮지만, 결측치가 많은 경우
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3)
figure.figure.set_size_inches(14,6)
sns.regplot(data=data, x='LowQualFinSF', y='SalePrice',ax=ax1)

sns.regplot(data=data, x='OpenPorchSF', y='SalePrice',ax=ax2)

sns.regplot(data=data, x='WoodDeckSF', y='SalePrice',ax=ax3)

In [15]:
# 이들은 0 값만 분리해주면 결과가 나쁘지않을수도 있기에 0만 분리
all_data['NoLowQual'] = (all_data['LowQualFinSF'] == 0)
all_data['NoOpenPorch'] =(all_data['OpenPorchSF'] == 0)
all_data['NoWoodDeck'] = (all_data['WoodDeckSF'] == 0)

In [16]:
# 전처리
# 1. 범주형 변수
# 범주형 변수가 상당히 많은 데이터 셋이다
# 범주형 변수를 다짜고짜 1,2,3,4... 로 인코딩해주면 머신러닝이 오해할 소지가 충분하기 때문에 모조리 원핫 인코딩을 한다
non_numeric = all_data.select_dtypes(np.object)

def onehot(col_list):
    global all_data
    while len(col_list) !=0:
        col = col_list.pop(0)
        data_encoded = pd.get_dummies(all_data[col], prefix=col)
        all_data = pd.merge(all_data, data_encoded, on="Id")
        all_data = all_data.drop(columns=col)
    print(all_data.shape)
    
onehot(list(non_numeric))

In [17]:
# 2. 수치형 변수
# 수치형 변수는 비대칭이 너무 심해지지 않게끔. 
# Right Skewed 가 크게 되어있는 데이터 들에만 로그를 씌워 적절히 변형
numeric = all_data.select_dtypes(np.number)

def log_transform(col_list):
    transformed_col = []
    while len(col_list)!=0:
        col=col_list.pop(0)
        if all_data[col].skew() > 0.5:
            all_data[col] = np.log(all_data[col] + 1)
            transformed_col.append(col)
        else:
            pass
    print(f"{len(transformed_col)} features had been transformed")
    print(all_data.shape)
    
log_transform(list(numeric))

In [18]:
print(train.shape, test.shape)
Xtrain = all_data[:len(train)]
Xtest = all_data[len(train):]
print(Xtrain.shape,Xtest.shape)

In [19]:
# 번외 추가
# 각 컬럼의 null 값에 해당하는 각 변수들의 평균으로 대체하는 라이브러리
# 평균값 대체를 위하여 scikit-learn의 Imputer 함수를 이용하였으며, strategy 값에 대체해주고자 하는 이름을 넣어주면 값 처리
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy='mean')
imputer.fit(boston_num_df)
boston_num_df_ = imputer.transform(boston_num_df)

boston_num_df = pd.DataFrame(boston_num_df_, columns=boston_num_df.columns, index=boston_df_index)